In [ ]:
import os

import lightning as L
import rootutils
import wandb
from hydra.utils import get_class, instantiate

rootutils.setup_root(os.path.abspath(""), pythonpath=True)

In [ ]:
api = wandb.Api()

run_id = ""
run = api.run(run_id)

In [ ]:
model_class = get_class(run.config["model"]["_target_"])
ckpt = run.config["callbacks"]["model_checkpoint"]["dirpath"] + "/best.ckpt"
model = model_class.load_from_checkpoint(ckpt)  # type: ignore

dm = instantiate(run.config["data"])
dm.prepare_data()
dm.setup()

In [ ]:
trainer = L.Trainer(logger=False)
metrics = trainer.test(model, datamodule=dm)

In [ ]:
run.summary.update({"extraMetrics": metrics[0]})